In [2]:
import pandas as pd

In [3]:
resource_file_path = "Resources/purchase_data.csv"
# Check the encoding
with open(resource_file_path) as f:
    print(f)

<_io.TextIOWrapper name='Resources/purchase_data.csv' mode='r' encoding='UTF-8'>


In [80]:
df = pd.read_csv(resource_file_path)
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [5]:
# Check if data needs to be cleaned
df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [98]:
# Count total number of players
total_players = len(df.groupby("SN"))
print(f"Total Players: \n{total_players}")

Total Players: 
576


### Purchasing Analysis (Total)
- Run basic calculations to obtain number of unique items, average price, etc.
- Create a summary data frame to hold the results
- Optional: give the displayed data cleaner formatting
- Display the summary data frame

In [108]:
# Number of Unique Items
unique_items_number = len(df["Item Name"].unique())

# Average Purchase Price
average_purchase_price = round(df["Price"].mean(),2)

# Total Number of Purchases
total_number_purchases = len(df["Purchase ID"])

# Total Revenue
total_revenue = df["Price"].sum()

purchasing_analysis = pd.DataFrame({
    "Number of Unique Items":[unique_items_number],
    "Average Price":[f"${average_purchase_price}"],
    "Number of Purchases":[total_number_purchases],
    "Total Revenue":[f"${total_revenue}"]
})
purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


### Gender Demographics
- Percentage and Count of Male Players
- Percentage and Count of Female Players
- Percentage and Count of Other / Non-Disclosed

In [159]:
gender = ["Female", "Male", "Other / Non-Disclosed"]

# Create function to count players by gender, argument specifies the gender
def count_players_by_gender(val):
    if val in [gender[0], gender[1]]:
        gender_df = df.loc[df['Gender'] == val]
    else: gender_df = df.loc[(df['Gender'] != gender[0])&(df['Gender'] != gender[1])
]
    gender_players_count = len(gender_df.groupby("SN"))
    return gender_players_count

def gender_percentage(count):
    return round(count/total_players*100, 2)

# Create Lists to store Gender Demographics related data
gender_count = [
    count_players_by_gender(gender[0]),
    count_players_by_gender(gender[1]),
    count_players_by_gender(gender[2])
]
gender_percentage = [
    f"{gender_percentage(gender_count[0])}%",
    f"{gender_percentage(gender_count[1])}%",
    f"{gender_percentage(gender_count[2])}%",
]

# Create DataFrame 
gender_demographics = pd.DataFrame({
    "Total Count":gender_count,
    "Percentage of Players":gender_percentage},
    index = gender)
gender_demographics

,Total Count,Percentage of Players
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


### Purchasing Analysis (Gender)
- Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
- Create a summary data frame to hold the results
- Optional: give the displayed data cleaner formatting
- Display the summary data frame

In [168]:
# The below each broken by gender
# Purchase Count
def count_by_gender(val):
    purchase_count_by_gender = df[["Gender", "Purchase ID"]].groupby(["Gender"]).count()
    return purchase_count_by_gender['Purchase ID'][val]

purchase_count = [
    count_by_gender(gender[0]), count_by_gender(gender[1]), count_by_gender(gender[2])
]

# Average Purchase Price
purchase_price_by_gender = df[["Gender", "Price"]].groupby(["Gender"]).sum()

def price_by_gender(val):  
    return round(purchase_price_by_gender['Price'][val]/purchase_count[gender.index(val)],2)

average_purchase = [
    price_by_gender(gender[0]), price_by_gender(gender[1]), price_by_gender(gender[2])
]

# Total Purchase Value
total_purchase = purchase_price_by_gender["Price"]

# Average Purchase Total per Person by Gender
# At first apply groupby by both columns - Gender and SD to find out the total purchase sum by each player
purchase_per_person = df[["Gender", "SN", "Price"]].groupby(["Gender", "SN"]).sum()

# Then use group by only by gender to find the average purshase total per person by gender
average_purchase_per_person = round(purchase_per_person.groupby("Gender").mean(), 2)

avg_total = average_purchase_per_person["Price"]

purchasing_analysis = pd.DataFrame({
    "Purchase Count": purchase_count,
    "Average Purchase Price": average_purchase,
    "Total Purchase Value": total_purchase,
    "Avg Total Purchase per Person": avg_total},
    index = gender
)
purchasing_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


### Age Demographics
- Establish bins for ages
- Categorize the existing players using the age bins. Hint: use pd.cut()
- Calculate the numbers and percentages by age group
- Create a summary data frame to hold the results
- Optional: round the percentage column to two decimal points
- Display Age Demographics Table

In [175]:
# The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
min_age = df["Age"].min()

max_age = df["Age"].max()

bins = [x for x in range(min_age-2, max_age+3, 5)]
print(bins)
bins_labels = ["<10", "11-15", "16-20", "21-25", "26-30",
               "31-35)", "36-40", "40+"]

# Purchase Count
purchase_by_age_bins = df
purchase_by_age_bins["Age Group"] = pd.cut(df["Age"], bins, labels = bins_labels)
purchase_count_by_age_bins = purchase_by_age_bins[["Age Group", "Purchase ID"]].groupby("Age Group").count()
print(f"\nPurchase Count by Age Bins: \n{purchase_count_by_age_bins}")

# Average Purchase Price
purchase_price_by_age_bins = round(purchase_by_age_bins[["Age Group", "Price"]].groupby("Age Group").mean(),2)
print(f"\nAverage Purchase Price by Age Bins: \n {purchase_price_by_age_bins}")

# Total Purchase Value
purchase_total_value_by_age_bins = round(purchase_by_age_bins[["Age Group", "Price"]].groupby("Age Group").sum(),2)
print(f"\nTotal Purchase Value by Age Bins: \n {purchase_total_value_by_age_bins}")

# Average Purchase Total per Person by Age Group
# As column Age Group is considered as category type, we need to get rid of the NaN values
purchase_sn = purchase_by_age_bins[["Age Group", "SN", "Purchase ID"]].groupby(["SN", "Age Group"]).count().dropna()
purchase_total_by_age_bins = round(purchase_sn.groupby("Age Group").mean(),2)
print(f"\nAverage Purchase Total per Person by Age Group: \n {purchase_total_by_age_bins}")

[5, 10, 15, 20, 25, 30, 35, 40, 45]

Purchase Count by Age Bins: 
           Purchase ID
Age Group             
<10                 32
11-15               54
16-20              200
21-25              325
26-30               77
31-35)              52
36-40               33
40+                  7

Average Purchase Price by Age Bins: 
            Price
Age Group       
<10         3.40
11-15       2.90
16-20       3.11
21-25       3.02
26-30       2.88
31-35)      2.99
36-40       3.40
40+         3.08

Total Purchase Value by Age Bins: 
             Price
Age Group        
<10        108.96
11-15      156.60
16-20      621.56
21-25      981.64
26-30      221.42
31-35)     155.71
36-40      112.35
40+         21.53

Average Purchase Total per Person by Age Group: 
            Purchase ID
Age Group             
<10               1.33
11-15             1.32
16-20             1.33
21-25             1.40
26-30             1.31
31-35)            1.41
36-40             1.27
40+               1.

In [79]:
### Top Spenders
# * Identify the top 5 spenders in the game by total purchase value, then list (in a table):
#   * SN
#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value

top_spenders = df.groupby("SN").agg({
    "Price": "sum",
    "Purchase ID": "count"}).sort_values(by=["Price"], ascending=False)
top_spenders["Average Purchase Price"] = round(df.groupby("SN")["Price"].mean(), 2)
top_spenders = top_spenders.rename(columns = {
    "Price": "Total Purchase Value",
    "Purchase ID": "Purchase Count"
})
print("Top 5 Spenders:")
top_spenders.head()

Top 5 Spenders:


,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,18.96,5,3.79
Idastidru52,15.45,4,3.86
Chamjask73,13.83,3,4.61
Iral74,13.62,4,3.40
Iskadarya95,13.10,3,4.37


In [ ]:
### Most Popular Items
# * Identify the 5 most popular items by purchase count, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value

top_items = df[["Item ID", "Item Name", "Purchase ID", "Price"]].groupby("Item Name").agg({
    "Item ID": "first",
    "Purchase ID": "count",
    "Price": "first"
})
top_items = top_items.rename(columns = {"Purchase ID": "Purchase Count","Price": "Item Price"})
top_items["Total Purchase Value"] = df.groupby("Item Name")["Price"].sum()
top_items = top_items.sort_values(by = ["Purchase Count"], ascending = False)

print("The 5 most popular items by purchase count:")
top_items.head(10)

In [77]:
### Most Profitable Items
# * Identify the 5 most profitable items by total purchase value, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value

profit_items = top_items.sort_values(by = ["Total Purchase Value"], ascending=False)
print("The 5 most profitable items by total purchase value:")
profit_items.head(10)

The 5 most profitable items by total purchase value:


,Item ID,Purchase Count,Item Price,Total Purchase Value
Item Name,,,,
Final Critic,92,13,4.88,59.99
"Oathbreaker, Last Hope of the Breaking Storm",178,12,4.23,50.76
Nirvana,82,9,4.90,44.10
Fiery Glass Crusader,145,9,4.58,41.22
Singed Scalpel,103,8,4.35,34.80
"Lightning, Etcher of the King",59,8,4.23,33.84
"Extraction, Quickblade Of Trembling Hands",108,9,3.53,31.77
"Glimmer, Ender of the Moon",78,7,4.40,30.80
Winter's Bite,72,8,3.77,30.16
